# prepare_dataset2

To prepare a new datatset for training the network for inverse kinematics.

## Step 1: Import modules

In [3]:
import os
import cv2
import sys
import glob
import time
import threading
import numpy as np
import pandas as pd
from zeno_face_tracker_helpers import *
try:
    from ConfigParser import ConfigParser    # If using Python 2.7
except ImportError:
    from configparser import ConfigParser    # If using Python 3.5
config = ConfigParser()
config.read('config.ini')
sys.path.append(os.path.realpath(config.get('zeno_interface', 'repository_path')))
from video_sources import *
from zeno_interface import *
sys.path.append(os.path.realpath(config.get('facial_landmark_tracker', 'repository_path')))
from ibug_face_tracker import *
from zeno_face_tracker import *
print('All modules imported.')

All modules imported.


## Step 2: Collect data

In [9]:
samples = []
zeno_head = None
video_source = None
samples_path = None
config.read('config.ini')
motor_duration = config.getint('zeno_interface', 'motor_duration')
grace_period = config.getint('zeno_interface', 'grace_period')
try:
    # Load configuration related to data collection
    batch_size = max(1, config.getint('prepare_dataset2', 'batch_size'))
    move_neck = min(max(0, config.getint('prepare_dataset2', 'move_neck')), 2)
    move_face = min(max(0, config.getint('prepare_dataset2', 'move_face')), 2)
    sampling_delay = config.getint('prepare_dataset2', 'sampling_delay') / 1000.0
    retry_limit = max(1, config.getint('prepare_dataset2', 'retry_limit'))
    if move_neck == 2:
        print('Neck joints will be sampled from N(0.5, 0.5/3).')
    elif move_neck == 1:
        print('Neck joints will be sampled from U(0.0, 1.0).')
    else:
        print('Neck joints will not be moved.')
    if move_face == 2:
        print('Facial actuators will be sampled from N(0.5, 0.5/3).')
    elif move_face == 1:
        print('Facial actuators will be sampled from U(0.0, 1.0).')
    else:
        print('Facial actuators will not be moved.')

    # Connect to the robot
    zeno_ip_address = config.get('zeno_interface', 'zeno_ip_address')
    zeno_head = ZenoHead(zeno_ip_address)
    print('Connected to zeno at %s.' % zeno_ip_address)
    
    # Initialise the tracker
    tracker68 = ZenoFaceTracker(os.path.realpath('./models/zeno_landmark_tracker_68.model'), 
                                os.path.realpath(config.get('facial_landmark_tracker', 'auxiliary_model_path')), 
                                os.path.realpath('./models/zeno_face_detector.model'))
    tracker68.failure_detection_interval = 1
    print('Lanmark tracker initialised.')

    # Open the video source
    video_source_id = config.get('video_source', 'video_source')
    frame_width = config.getint('video_source', 'frame_width')
    frame_height = config.getint('video_source', 'frame_height')
    frame_rate = config.getfloat('video_source', 'frame_rate')
    if video_source_id == 'window':
        window_title_re = config.get('window_specification', 'title_re')
        window_class_name = config.get('window_specification', 'class_name')
        child_identifier = config.get('window_specification', 'child_identifier')
        window_roi = config.get('window_specification', 'window_roi').replace(
            '\'', '').replace('\"', '').replace('\t', '').replace(' ', '')
        window_roi = tuple([float(x) for x in window_roi.split(',') if len(x) > 0])
        pywinauto = importlib.import_module('pywinauto')
        top_level_window = pywinauto.Desktop(backend='win32').window(title_re=window_title_re,
                                                                     class_name=window_class_name,
                                                                     visible_only=False)
        if len(child_identifier) > 0:
            window_handle = top_level_window[child_identifier].handle
        else:
            window_handle = top_level_window.handle
            video_source = ThreadedWindowCapture(window_handle, frame_width, frame_height, 
                                                 frame_rate, window_roi)
            print('Window capture initialised.')
    else:
        video_source = ThreadedWebcam(int(video_source_id), frame_width, frame_height, frame_rate)
        print('Webcam #%d opened.' % int(video_source_id))
    
    # Create the recording folder
    dataset_folder = os.path.realpath('./dataset2')
    if not os.path.isdir(dataset_folder):
        os.makedirs(dataset_folder)
    batch_folder = os.path.join(dataset_folder, 'batch_%03d' % len(glob.glob(os.path.join(dataset_folder, 'batch_*'))))
    os.makedirs(batch_folder)
    rendering_folder = os.path.join(batch_folder, 'renderings')
    os.makedirs(rendering_folder)
    samples_path = os.path.join(batch_folder, 'samples.pkl')
    print('All %d samples will be saved to: ' % batch_size + batch_folder)
    
    # The processing loop
    actuator_values = default_position = zeno_head.get_default_motor_positions()
    cv2.namedWindow('68 landmark model', cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_KEEPRATIO)
    cv2.resizeWindow('68 landmark model', (frame_width, frame_height))
    zeno_head.move(actuator_values, motor_duration)
    last_move_time = time.time()
    retries = 0
    while cv2.getWindowProperty('68 landmark model', 0) >= 0 and len(samples) < batch_size:
        is_new_frame, frame = video_source.read(0.01)
        if is_new_frame:
            if frame is None:
                break
            else:
                tracker68.track(frame)
                rendering = frame.copy()
                tracker68.plot_current_result(rendering)
                cv2.imshow('68 landmark model', rendering)
                time_elapsed = time.time() - last_move_time
                if time_elapsed >= sampling_delay:
                    if tracker68.has_facial_landmarks:
                        # Draw a sample!
                        sample = {}
                        sample['landmarks'] = tracker68.facial_landmarks
                        if tracker68.has_eye_landmarks:
                            sample['eye_points'] = tracker68.eye_landmarks
                        if tracker68.has_fitting_scores:
                            sample['fitting_scores'] = tracker68.most_recent_fitting_scores
                        if tracker68.has_head_pose:
                            sample['head_pose'] = (tracker68.pitch, tracker68.yaw, tracker68.roll)
                        sample['actuator_values'] = actuator_values[:]
                        sample['actuator_values_reading'] = zeno_head.get_motor_positions()
                        frame_path = os.path.join(batch_folder, 'frame_%06d.png' % len(samples))
                        cv2.imwrite(frame_path, frame)
                        rendering_path = os.path.join(rendering_folder, 'frame_%06d.jpg' % len(samples))
                        cv2.imwrite(rendering_path, rendering)
                        samples.append(sample)
                        print('Sample %d of %d: ' % (len(samples), batch_size) + 
                              ', '.join(['%.6f' % x for x in actuator_values]))
                    else:
                        retries = retries + 1
                    if tracker68.has_facial_landmarks or retries >= retry_limit:
                        # Move to a new position
                        if move_neck == 2:
                            actuator_values[0:2] = np.clip(np.random.normal(0.5, 0.5 / 3, 2), 0.0, 1.0).tolist()
                        elif move_neck == 1:
                            actuator_values[0:2] = np.random.uniform(0.0, 1.0, 2).tolist()
                        else:
                            actuator_values[0:2] = default_position[0:2]
                        if move_face == 2:
                            actuator_values[2:] = np.clip(np.random.normal(0.5, 0.5 / 3, 5), 0.0, 1.0).tolist()
                        elif move_face == 1:
                            actuator_values[2:] = np.random.uniform(0.0, 1.0, 5).tolist()
                        else:
                            actuator_values[2:] = default_position[2:]
                        zeno_head.move(actuator_values, motor_duration)
                        last_move_time = time.time()
                elif time_elapsed >= motor_duration / 1000.0:
                    # Hold current position
                    zeno_head.move(actuator_values, 40)
        key = cv2.waitKey(10)
        if key == ord('q') or key == ord('Q'):
            break
finally:
    cv2.destroyAllWindows()
    if len(samples) > 0 and samples_path is not None:
        pd.DataFrame(samples).to_pickle(samples_path, protocol=2)
        print('All sample records are saved to: ' + samples_path)
    if video_source is not None:
        video_source.release()
    if zeno_head is not None:
        zeno_head.reset(motor_duration)
        time.sleep(max(motor_duration, grace_period) / 1000.0)
        zeno_head.disconnect()
    print('We are done here.')

Neck joints will be sampled from U(0.0, 1.0).
Facial actuators will be sampled from U(0.0, 1.0).
Connected to zeno at 127.0.0.1.
Lanmark tracker initialised.
Webcam #0 opened.
All 1024 samples will be saved to: D:\hhj\zeno_face_tracker\dataset2\batch_009
Sample 1 of 1024: 0.502370, 0.778409, 0.502041, 0.872951, 0.500000, 0.501736, 0.157534
Sample 2 of 1024: 0.250398, 0.270581, 0.610537, 0.491132, 0.976848, 0.596505, 0.622939
Sample 3 of 1024: 0.845912, 0.876380, 0.841425, 0.375487, 0.921695, 0.872206, 0.316071
Sample 4 of 1024: 0.430863, 0.777826, 0.635679, 0.945937, 0.769086, 0.292797, 0.378895
Sample 5 of 1024: 0.763265, 0.308399, 0.100418, 0.360030, 0.806505, 0.789986, 0.102466
Sample 6 of 1024: 0.382721, 0.438438, 0.254251, 0.765518, 0.462410, 0.239800, 0.566810
Sample 7 of 1024: 0.701813, 0.851029, 0.552984, 0.863695, 0.829483, 0.626629, 0.260540
Sample 8 of 1024: 0.698498, 0.008787, 0.751152, 0.873440, 0.070678, 0.366522, 0.848227
Sample 9 of 1024: 0.435505, 0.104208, 0.913124, 0

Sample 92 of 1024: 0.642171, 0.973938, 0.860915, 0.235041, 0.145019, 0.165125, 0.751983
Sample 93 of 1024: 0.322942, 0.664139, 0.250567, 0.283311, 0.618382, 0.080312, 0.181694
Sample 94 of 1024: 0.901792, 0.591980, 0.754740, 0.168118, 0.249512, 0.984913, 0.032641
Sample 95 of 1024: 0.731003, 0.721306, 0.921448, 0.060744, 0.530438, 0.578234, 0.912845
Sample 96 of 1024: 0.498574, 0.146990, 0.414338, 0.789406, 0.023298, 0.159992, 0.360814
Sample 97 of 1024: 0.229623, 0.722786, 0.636412, 0.287415, 0.345064, 0.562658, 0.663385
Sample 98 of 1024: 0.207094, 0.636462, 0.210743, 0.207906, 0.746143, 0.631256, 0.686965
Sample 99 of 1024: 0.901175, 0.888909, 0.370705, 0.296946, 0.661338, 0.858491, 0.348702
Sample 100 of 1024: 0.702558, 0.352444, 0.840864, 0.020065, 0.460507, 0.635661, 0.929206
Sample 101 of 1024: 0.763199, 0.432174, 0.400172, 0.014974, 0.506298, 0.265758, 0.688121
Sample 102 of 1024: 0.264776, 0.203995, 0.267658, 0.868722, 0.032807, 0.533466, 0.544960
Sample 103 of 1024: 0.185076,

Sample 185 of 1024: 0.304420, 0.999050, 0.306520, 0.772013, 0.065468, 0.993236, 0.915413
Sample 186 of 1024: 0.625978, 0.601220, 0.887946, 0.047844, 0.266968, 0.261638, 0.975777
Sample 187 of 1024: 0.478277, 0.447550, 0.164239, 0.816953, 0.868889, 0.999998, 0.446603
Sample 188 of 1024: 0.210212, 0.663797, 0.057843, 0.567947, 0.336212, 0.251896, 0.437098
Sample 189 of 1024: 0.313691, 0.464258, 0.192291, 0.733113, 0.322645, 0.744407, 0.690017
Sample 190 of 1024: 0.755210, 0.608735, 0.659398, 0.642103, 0.895764, 0.585498, 0.887546
Sample 191 of 1024: 0.904682, 0.648390, 0.641316, 0.339076, 0.173920, 0.181228, 0.158224
Sample 192 of 1024: 0.253221, 0.573703, 0.937224, 0.755564, 0.043954, 0.511658, 0.989868
Sample 193 of 1024: 0.501702, 0.117406, 0.094638, 0.022629, 0.856131, 0.509670, 0.438729
Sample 194 of 1024: 0.493184, 0.562404, 0.794301, 0.196366, 0.766826, 0.434600, 0.511354
Sample 195 of 1024: 0.370016, 0.725051, 0.321339, 0.072763, 0.820762, 0.533790, 0.508876
Sample 196 of 1024: 0

Sample 278 of 1024: 0.254619, 0.071858, 0.653845, 0.268177, 0.700875, 0.216657, 0.624648
Sample 279 of 1024: 0.940911, 0.734691, 0.764024, 0.291154, 0.865337, 0.884148, 0.992830
Sample 280 of 1024: 0.921511, 0.696673, 0.652125, 0.061222, 0.825876, 0.792810, 0.461050
Sample 281 of 1024: 0.469471, 0.803463, 0.808484, 0.446991, 0.961836, 0.275346, 0.045086
Sample 282 of 1024: 0.211289, 0.277324, 0.089772, 0.288038, 0.950340, 0.325237, 0.765730
Sample 283 of 1024: 0.234991, 0.543787, 0.605683, 0.129942, 0.156529, 0.506066, 0.832957
Sample 284 of 1024: 0.167982, 0.334252, 0.560584, 0.025132, 0.168079, 0.431209, 0.818368
Sample 285 of 1024: 0.262379, 0.625370, 0.707927, 0.674183, 0.910658, 0.913215, 0.599422
Sample 286 of 1024: 0.714915, 0.261932, 0.915781, 0.456049, 0.622100, 0.977109, 0.165000
Sample 287 of 1024: 0.212867, 0.614289, 0.033203, 0.225451, 0.307536, 0.969431, 0.446263
Sample 288 of 1024: 0.580367, 0.583544, 0.966527, 0.747296, 0.220427, 0.886269, 0.412532
Sample 289 of 1024: 0

Sample 371 of 1024: 0.154866, 0.720401, 0.552853, 0.517985, 0.203389, 0.094128, 0.742959
Sample 372 of 1024: 0.874721, 0.520648, 0.996388, 0.552874, 0.832575, 0.571621, 0.442569
Sample 373 of 1024: 0.211702, 0.469658, 0.700804, 0.496925, 0.995810, 0.434623, 0.122934
Sample 374 of 1024: 0.779238, 0.281939, 0.967104, 0.113969, 0.334922, 0.337973, 0.570126
Sample 375 of 1024: 0.801850, 0.308886, 0.775939, 0.333633, 0.854373, 0.438249, 0.638924
Sample 376 of 1024: 0.256203, 0.181911, 0.809133, 0.235597, 0.001466, 0.309422, 0.107933
Sample 377 of 1024: 0.843613, 0.400761, 0.715110, 0.517686, 0.269278, 0.706213, 0.156121
Sample 378 of 1024: 0.570164, 0.011965, 0.077418, 0.494473, 0.478156, 0.371131, 0.029803
Sample 379 of 1024: 0.649418, 0.904474, 0.682359, 0.015932, 0.017127, 0.168515, 0.580400
Sample 380 of 1024: 0.320566, 0.826977, 0.469558, 0.751338, 0.564362, 0.872908, 0.322388
Sample 381 of 1024: 0.149187, 0.270964, 0.536910, 0.080244, 0.777632, 0.189816, 0.994809
Sample 382 of 1024: 0

...

Sample 650 of 1024: 0.788205, 0.426291, 0.790611, 0.482261, 0.193338, 0.586567, 0.645633
Sample 651 of 1024: 0.485918, 0.537471, 0.187377, 0.259539, 0.295290, 0.520160, 0.127342
Sample 652 of 1024: 0.458747, 0.897248, 0.473433, 0.210019, 0.422010, 0.350889, 0.158475
Sample 653 of 1024: 0.429993, 0.116511, 0.012557, 0.248021, 0.926360, 0.077227, 0.820618
Sample 654 of 1024: 0.770071, 0.384312, 0.922877, 0.701924, 0.547840, 0.338917, 0.545314
Sample 655 of 1024: 0.855831, 0.754445, 0.350846, 0.468923, 0.035974, 0.858504, 0.536742
Sample 656 of 1024: 0.520436, 0.621086, 0.175368, 0.139629, 0.776016, 0.126216, 0.339343
Sample 657 of 1024: 0.895610, 0.396348, 0.434038, 0.576612, 0.977091, 0.256152, 0.623708
Sample 658 of 1024: 0.295285, 0.826863, 0.159346, 0.188069, 0.565290, 0.256490, 0.399846
Sample 659 of 1024: 0.601471, 0.174934, 0.538636, 0.129569, 0.250227, 0.642213, 0.893323
Sample 660 of 1024: 0.123330, 0.372445, 0.329246, 0.404913, 0.107034, 0.141455, 0.093200
Sample 661 of 1024: 0

Sample 743 of 1024: 0.594299, 0.039144, 0.759807, 0.121115, 0.607976, 0.139223, 0.544938
Sample 744 of 1024: 0.321176, 0.902766, 0.978776, 0.781312, 0.166495, 0.899923, 0.995407
Sample 745 of 1024: 0.701746, 0.042610, 0.112756, 0.308967, 0.722693, 0.253755, 0.782729
Sample 746 of 1024: 0.552941, 0.818136, 0.728866, 0.031326, 0.521068, 0.855739, 0.528015
Sample 747 of 1024: 0.183546, 0.707219, 0.351877, 0.138619, 0.455910, 0.161937, 0.144228
Sample 748 of 1024: 0.800953, 0.296528, 0.726740, 0.112850, 0.696859, 0.016852, 0.927839
Sample 749 of 1024: 0.365469, 0.297402, 0.416925, 0.819418, 0.183422, 0.699631, 0.852503
Sample 750 of 1024: 0.592430, 0.732002, 0.105613, 0.173127, 0.658636, 0.261567, 0.483019
Sample 751 of 1024: 0.489478, 0.152240, 0.028920, 0.247860, 0.110118, 0.386397, 0.099141
Sample 752 of 1024: 0.570815, 0.649885, 0.632199, 0.546420, 0.464758, 0.600532, 0.818180
Sample 753 of 1024: 0.774332, 0.748835, 0.584813, 0.007077, 0.966703, 0.904940, 0.383405
Sample 754 of 1024: 0

Sample 836 of 1024: 0.418313, 0.594765, 0.485244, 0.494340, 0.804442, 0.874256, 0.008746
Sample 837 of 1024: 0.629998, 0.369432, 0.188235, 0.374842, 0.467088, 0.322912, 0.265659
Sample 838 of 1024: 0.954529, 0.931167, 0.931254, 0.053909, 0.097957, 0.586720, 0.115747
Sample 839 of 1024: 0.878764, 0.201950, 0.706501, 0.167104, 0.179205, 0.951048, 0.817551
Sample 840 of 1024: 0.828257, 0.565095, 0.425059, 0.075874, 0.047981, 0.894750, 0.148793
Sample 841 of 1024: 0.369394, 0.740707, 0.978010, 0.112043, 0.475953, 0.111703, 0.548107
Sample 842 of 1024: 0.419108, 0.246164, 0.826566, 0.502981, 0.449331, 0.138989, 0.738733
Sample 843 of 1024: 0.745824, 0.121247, 0.283508, 0.758189, 0.651432, 0.201809, 0.210169
Sample 844 of 1024: 0.178917, 0.370714, 0.063898, 0.540606, 0.006841, 0.129187, 0.581651
Sample 845 of 1024: 0.371170, 0.934988, 0.145685, 0.279158, 0.560020, 0.471139, 0.270728
Sample 846 of 1024: 0.641612, 0.685386, 0.767800, 0.425305, 0.671786, 0.863401, 0.514328
Sample 847 of 1024: 0

Sample 929 of 1024: 0.878313, 0.956608, 0.680971, 0.630323, 0.642429, 0.497539, 0.027559
Sample 930 of 1024: 0.560336, 0.991824, 0.145969, 0.896948, 0.828558, 0.375425, 0.924191
Sample 931 of 1024: 0.764383, 0.586681, 0.871182, 0.459873, 0.788020, 0.645387, 0.175168
Sample 932 of 1024: 0.534059, 0.361353, 0.838664, 0.544374, 0.544383, 0.399910, 0.908211
Sample 933 of 1024: 0.750840, 0.289476, 0.112108, 0.831492, 0.990704, 0.186403, 0.414853
Sample 934 of 1024: 0.872620, 0.801168, 0.435934, 0.487620, 0.982133, 0.174995, 0.273667
Sample 935 of 1024: 0.161677, 0.015037, 0.092278, 0.569939, 0.647770, 0.793849, 0.171902
Sample 936 of 1024: 0.165900, 0.366563, 0.710814, 0.510819, 0.932569, 0.237157, 0.078100
Sample 937 of 1024: 0.705926, 0.349753, 0.832409, 0.982524, 0.954726, 0.296522, 0.379512
Sample 938 of 1024: 0.255282, 0.958679, 0.128060, 0.339960, 0.922362, 0.256750, 0.333963
Sample 939 of 1024: 0.206074, 0.940743, 0.964260, 0.455424, 0.629522, 0.418035, 0.041132
Sample 940 of 1024: 0

Sample 1021 of 1024: 0.345058, 0.724888, 0.405987, 0.183271, 0.533448, 0.827824, 0.883719
Sample 1022 of 1024: 0.307603, 0.638289, 0.194262, 0.702107, 0.882061, 0.395170, 0.971399
Sample 1023 of 1024: 0.385214, 0.628696, 0.730795, 0.679500, 0.689556, 0.164987, 0.529403
Sample 1024 of 1024: 0.299445, 0.084085, 0.891533, 0.250330, 0.661926, 0.602144, 0.528949
All sample records are saved to: D:\hhj\zeno_face_tracker\dataset2\batch_009\samples.pkl
We are done here.


## Step 3: Combine ERT (dlib) and AAM results and update renderings

__Run this only after producing the AAM results!__

In [11]:
config.read('config.ini')

# Enumerate the batches
dataset_folder = os.path.realpath('./dataset2')
batch_folders = glob.glob(os.path.join(dataset_folder, 'batch_*'))
batch_folders = [x for x in batch_folders if os.path.exists(os.path.join(x, 'samples.pkl')) and 
                 os.path.exists(os.path.join(x, 'aam_results.pkl')) and 
                 not os.path.exists(os.path.join(x, 'combined_samples.pkl'))]
print('%d batches to be processed.' % len(batch_folders))

# Process the batches
chehra_util = AuxiliaryUtility(os.path.realpath(config.get('facial_landmark_tracker', 'auxiliary_model_path')))
for batch_folder in batch_folders:
    print('Now processing: ' + batch_folder)
    combined_samples = []
    samples = pd.read_pickle(os.path.join(batch_folder, 'samples.pkl'))
    aam_results = pd.read_pickle(os.path.join(batch_folder, 'aam_results.pkl'))
    last_check_time = time.time()
    for idx in range(samples.shape[0]):
        sample = samples.iloc[idx].to_dict()
        sample['aam_landmarks'] = aam_results.iloc[idx]['landmarks']
        frame = cv2.imread(os.path.join(batch_folder, 'frame_%06d.png' % idx))
        sample['aam_eye_points'], sample['aam_fitting_scores'] = chehra_util.apply_additional_svrs(
            cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), sample['aam_landmarks'])
        sample['aam_head_pose'] = chehra_util.estimate_head_pose(sample['aam_landmarks'])
        combined_samples.append(sample)
        FaceTracker.plot_landmark_connections(frame, sample['aam_landmarks'])
        FaceTracker.plot_facial_landmarks(frame, sample['aam_landmarks'])
        FaceTracker.plot_eye_landmarks(frame, sample['aam_eye_points'])
        FaceTracker.plot_head_pose(frame, sample['aam_head_pose'][0], 
                                   sample['aam_head_pose'][1], sample['aam_head_pose'][2])
        rendering_path = os.path.join(batch_folder, 'aam_renderings', 'frame_%06d.jpg' % idx)
        cv2.imwrite(rendering_path, frame)
        if time.time() - last_check_time > 10.0:
            print('%d AAM renderings have been updated.' % len(combined_samples))
            last_check_time = time.time()
    print('All %d AAM renderings have been updated.' % len(combined_samples))
    combined_samples_path = os.path.join(batch_folder, 'combined_samples.pkl')
    pd.DataFrame(combined_samples).to_pickle(combined_samples_path, protocol=2)
    print('Combined samples are saved to: ' + combined_samples_path)
print('All done.')

2 batches to be processed.
Now processing: D:\hhj\zeno_face_tracker\dataset2\batch_008
177 AAM renderings have been updated.
342 AAM renderings have been updated.
508 AAM renderings have been updated.
679 AAM renderings have been updated.
856 AAM renderings have been updated.
All 1024 AAM renderings have been updated.
Combined samples are saved to: D:\hhj\zeno_face_tracker\dataset2\batch_008\combined_samples.pkl
Now processing: D:\hhj\zeno_face_tracker\dataset2\batch_009
179 AAM renderings have been updated.
356 AAM renderings have been updated.
533 AAM renderings have been updated.
713 AAM renderings have been updated.
881 AAM renderings have been updated.
All 1024 AAM renderings have been updated.
Combined samples are saved to: D:\hhj\zeno_face_tracker\dataset2\batch_009\combined_samples.pkl
All done.
